In [ ]:
# Initial setup
import pandas as pd
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')

df_full = pd.read_json('yelp_academic_dataset_business.json', lines = True)
tip = pd.read_json('yelp_academic_dataset_tip.json', lines = True)

## 1. Data Preprocessing
In this section, we will transform the Yelp dataset and produce the format required for data mining and modeling through the following steps:

- Filter to U.S. restaurants that were opened in 2016
- Flatten attributes and categories
- Transform categorical variables into dummy variables
- Handling missing values

In [ ]:
# Filter to U.S. restaurants that were opened in 2016
# 1) Filter to U.S. businesses
zip_crosswalk = pd.read_csv('zip_crosswalk.csv')
zip_crosswalk = zip_crosswalk.drop_duplicates(subset = ['ZIP', 'STATE'], keep = False)[['ZIP', 'STATE']]
# Transform digit-only postal code to int
digit_only = df_full['postal_code'].map(lambda x: True if all(i.isdigit() for i in x) else False)
df_full = df_full[(digit_only) & (df_full['postal_code'] != '')]
df_full['postal_code_int'] = [int(x) for x in df_full['postal_code']]
# Join crosswalk by zip and state
df_us = pd.merge(df_full, zip_crosswalk[['ZIP', 'STATE']], 
              left_on = ['postal_code_int', 'state'],
              right_on = ['ZIP', 'STATE'])

# 2) Filter to restaurants
df_us = df_us[df_us['categories'].str.contains('Restaurants|Food', na = False)]
df_us.set_index('business_id')

# 3) Filter to restaurants that were opened in 2016
start = '2016-01-01'
end = '2016-12-31'
tip = tip.groupby('business_id')['date'].min().reset_index()
tip = tip[(tip['date'] >= start) & (tip['date'] <= end)]
df = pd.merge(tip, df_us, on = 'business_id', how = 'inner')

In [ ]:
# Check geological distribution
import seaborn as sns
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(10, 5))
df.groupby('state')['business_id'].count().plot(kind = 'bar', color = '#0066cc', width = 0.8, linewidth = 1, edgecolor = 'black')
plt.xlabel('U.S. State', fontsize = 15)
plt.ylabel('Restaurant Count', fontsize = 15)
plt.xticks(fontsize = 15, rotation = 0)
plt.show()

In [ ]:
# Drop states with few data points
state_list = ['AZ', 'NC', 'NV', 'OH', 'PA', 'WI']
print('Total data points in df:', len(df))
df = df[np.in1d(df['state'], state_list)].reset_index()
print('Total data points in df after removing a few states:', len(df))

In [ ]:
# Check restaurant closure
df.groupby('is_open')['business_id'].count().plot(kind = 'bar')
plt.show()

In [ ]:
# Flatten attributes and categories
# 1) Categories
from collections import Counter
# Extract categories from list of lists
cat = ', '.join(df['categories'])
# Count frequency
counter = Counter(cat.split(', '))
# Append categories as dummies
for cat in list(counter.keys()):
    df[cat] = np.where(df['categories'].str.contains(cat), 1, 0)
df.drop('categories', inplace = True, axis = 1)

In [ ]:
# 2) Attributes
# Unuseful attributes to be dropped
drop_list = ['AcceptsInsurance', 'AgesAllowed', 'Ambience', 'DietaryRestrictions',
             'BestNights', 'BikeParking', 'BusinessAcceptsBitcoin', 'Music', 'ByAppointmentOnly',
             'CoatCheck', 'DogsAllowed' , 'DriveThru', 'GoodForDancing', 'Smoking', 'BusinessParking', 'GoodForMeal']
# Flattening
att = df['attributes'].apply(pd.Series)
meal = att['GoodForMeal'].fillna("{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'breakfast': False, 'brunch': False}")
parking = att['BusinessParking'].fillna("{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}")
parking_list = ['garage', 'street', 'validated', 'lot', 'valet']
meal_list = ['dessert', 'latenight', 'lunch', 'dinner', 'breakfast', 'brunch']

# Define a function to flatten json lists
import ast
def flatten_json(df, col):
    for i in range(len(col)):
        json_dict = ast.literal_eval(col[i])
        for j in json_dict.keys():
            df[j] = np.where(json_dict[j], 1, 0) 
            
flatten_json(df = att, col = parking)
flatten_json(df = att, col = meal)

# Handle missing value in attributes
att['Alcohol'].fillna('none', inplace = True)
att['NoiseLevel'].fillna('average', inplace = True)
att['RestaurantsAttire'].fillna('casual', inplace = True)
att['WiFi'].fillna('no', inplace = True)
att['RestaurantsPriceRange2'].fillna(0, inplace = True)

# Transform attributes into dummy variables
att = att.drop(drop_list, axis = 1).fillna(value = False)
att = pd.get_dummies(att, drop_first = True)

# Append attributes to main dataframe
df = pd.merge(df, att, left_index = True, right_index = True)
df.drop('attributes', axis = 1, inplace = True)

## 2. External Data
In this section, we will integrate the following external data sources to the Yelp dataset:
- Zillow property price data
- Demographic data by zip code

In [ ]:
# Zillow
zillow = pd.read_csv('zillow_median_price.csv', encoding='cp1252')
# Extract zipcodes
zipcodes = df['postal_code'].unique()
df['postal_code_int'] = [int(i) for i in df['postal_code']]
house_price = zillow[np.in1d(zillow['RegionName'], 
                             df['postal_code_int'])][['RegionName', '2016-01']]
house_price.columns = ['postal_code_int', 'median_sqft_price']
# Append median house price to main df
df = pd.merge(df, house_price, on = 'postal_code_int', how = 'left')
df['median_sqft_price'] = df['median_sqft_price'].fillna(df['median_sqft_price'].median())

In [ ]:
# Add zip-level demographics
from uszipcode import SearchEngine
keys = ['zipcode', 'housing_units', 'land_area_in_sqmi', 'median_home_value', 'median_household_income', 
        'occupied_housing_units', 'population', 'population_density', 'annual_individual_earnings', 
        'educational_attainment_for_population_25_and_over', 'employment_status', 'families_vs_singles', 
        'households_with_kids', 'housing_occupancy', 'means_of_transportation_to_work_for_workers_16_and_over', 
        'population_by_age', 'population_by_gender', 'population_by_race', 'travel_time_to_work_in_minutes']
keep_col = ['housing_units', 'land_area_in_sqmi', 'median_home_value', 'median_household_income', 
            'occupied_housing_units', 'population', 'population_density', 'postal_code']
lst = []
for zipcode in zipcodes:
    search = SearchEngine(simple_zipcode=False) 
    item = search.by_zipcode(zipcode)
    newDict  = item.to_dict()
    lst.append([newDict.get(key) for key in keys])
keys[0] = 'postal_code'
df_demograph = pd.DataFrame(lst, columns= keys)[keep_col]
df = pd.merge(df, df_demograph, on = 'postal_code', how='inner')

## 3. Feature Engineering
In this section, we will perform feature engineering to extract the following attributes:
- Restaurant density and competition level in the neighborhood
- Whether a given restaurant belong to a local/national chain

In [ ]:
# Restaurant density
import geopy.distance
business_id = []
density = []
avg_stars = []
avg_review_count = []
std_stars = []
std_review_count = []

# Create a random sample from all U.S. restaurants (n = 10000) to calculate restaurant density
df_us = df_us[np.in1d(df_us['state'], state_list)]
df_us = df_us.dropna(subset = ['latitude', 'longitude'], axis = 0).reset_index()
sample_us = df_us.drop_duplicates(subset=['latitude', 'longitude', 'stars', 'review_count']).sample(n = 10000).reset_index()

In [ ]:
for i in range(len(df)):
    print('Looping over', i, '...')
    coord1 = [df.at[i, 'latitude'], df.at[i, 'longitude']]
    count = 1
    stars = 0
    review_count = 0
    for j in range(len(sample_us)):
        coord2 = [sample_us.at[j, 'latitude'], sample_us.at[j, 'longitude']]
        try:
            distance = geopy.distance.vincenty(coord1, coord2).miles
            if distance < 1:
                count = count + 1
                stars = stars + sample_us.at[j, 'stars']
                review_count = review_count + sample_us.at[j, 'review_count']
        except:
            print('Error!')
    business_id.append(df.at[i, 'business_id'])
    density.append(count)
    avg_stars.append(stars/count)
    avg_review_count.append(review_count/count)

In [ ]:
biz_density = pd.DataFrame({'business_id': business_id,
                            'density' : density,
                            'avg_stars' : avg_stars,
                            'avg_review_count' : avg_review_count})
df = pd.merge(df, biz_density, on = 'business_id')

In [ ]:
# Identify whether a given restaurant is a local or national chain
local_chain = df.groupby(['state', 'name'])['business_id'].count().reset_index()
local_chain.columns = ['state', 'name', 'count']
local_chain['is_local_chain'] = np.where(local_chain['count'] > 1, 1, 0)

national_chain = local_chain.groupby('name')['state'].count().reset_index()
national_chain.columns = ['name', 'count']
national_chain['is_national_chain'] = np.where(national_chain['count'] > 2, 1, 0)

df = pd.merge(df, local_chain[['name', 'state', 'is_local_chain']], on = ['state','name'], how = 'left')
df = pd.merge(df, national_chain[['name', 'is_national_chain']], on = ['name'], how = 'left')

In [ ]:
# Data Cleaning
# pd.set_option("display.max_columns",500)
# Drop non-features
drop_col = ['index', 'business_id', 'date', 'address', 'hours', 
            'latitude','longitude','name','review_count','stars',
            'neighborhood', 'postal_code', 'postal_code_int', 'ZIP', 'STATE']
# Drop zipcide with missing value
df_clean = df[(df['postal_code'] != '89158') & (df['postal_code'] != '85378')].drop(drop_col, axis = 1)

In [ ]:
# Convert categorical variables to dummies
dummy_col = ['city', 'state']
df_clean = pd.get_dummies(data = df_clean, columns = dummy_col).dropna(how = 'any')

### 4. Modeling

In [4]:
# Resampling
df_clean.groupby('is_open')['is_open'].count()

is_open
0     485
1    2324
Name: is_open, dtype: int64

In [19]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import RobustScaler

X = df_clean.drop('is_open', axis = 1)
y = np.where(df_clean['is_open'] == 1, 0, 1)

# Scale Transform X
cols = X.columns
trans = RobustScaler().fit(X)
X = pd.DataFrame(trans.transform(X))
X.columns = cols

# Perform stratified 4-fold
k = 4
skf = StratifiedKFold(n_splits = k, shuffle = True, random_state = 123)

#### 4.1 Logistic regression (baseline)

In [ ]:
# Baseline: logitstic regression with regularization + grid search
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from sklearn import metrics

auc = []
pr_auc = []

for train_index, test_index in skf.split(X,y):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    y_train, y_test = y[train_index], y[test_index]
    
    # Upsample minority class
    X_close = X_train.iloc[y_train == 1, :]
    X_open = X_train.iloc[y_train == 0, :]
    y_close = y_train[y_train == 1]
    y_open = y_train[y_train == 0]
    X_close, y_close = resample(X_close, y_close, n_samples = len(X_open), random_state = 123)
    X_train = pd.concat([X_close, X_open])
    y_train = np.concatenate((y_close, y_open))
    
    lr = LogisticRegression()
    lr.fit(X_train, y_train)
    y_pred = lr.predict_proba(X_test)
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred[:, 1])
    auc.append(metrics.auc(fpr, tpr))
    pr_auc.append(metrics.average_precision_score(y_test, y_pred[:, 1]))

print('The average PR-AUC is', np.mean(pr_auc))
print('The standard deviation is', np.std(pr_auc))
print('The average AUC is', np.mean(auc))
print('The standard deviation is', np.std(auc))

In [ ]:
# Logistic regression with SMOTE
from imblearn.over_sampling import SMOTE
auc = []
pr_auc = []

for train_index, test_index in skf.split(X,y):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    y_train, y_test = y[train_index], y[test_index]
    
    # Upsample minority class in training data using SMOTE
    X_train, y_train = SMOTE(random_state = 123).fit_resample(X_train, y_train)
    
    lr = LogisticRegression()
    lr.fit(X_train, y_train)
    y_pred = lr.predict_proba(X_test)
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred[:, 1])
    auc.append(metrics.auc(fpr, tpr))
    pr_auc.append(metrics.average_precision_score(y_test, y_pred[:, 1]))

print('The average PR-AUC is', np.mean(pr_auc))
print('The standard deviation is', np.std(pr_auc))
print('The average AUC is', np.mean(auc))
print('The standard deviation is', np.std(auc))

In [ ]:
#### Logistic regression with regularization grid search
auc = []
pr_auc = []
penalty = []
reg = []

for train_index, test_index in skf.split(X,y):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    y_train, y_test = y[train_index], y[test_index]
    
    # Upsample minority class in training data
    X_close = X_train.iloc[y_train == 1, :]
    X_open = X_train.iloc[y_train == 0, :]
    y_close = y_train[y_train == 1]
    y_open = y_train[y_train == 0]
    X_close, y_close = resample(X_close, y_close, n_samples = len(X_open))
    X_train = pd.concat([X_close, X_open])
    y_train = np.concatenate((y_close, y_open))
    
    for p in ['l1', 'l2']:
        for c in [10**i for i in range(-3, 3)]:
            lr = LogisticRegression(penalty = p, C = c)
            lr.fit(X_train, y_train)
            y_pred = lr.predict_proba(X_test)
            fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred[:, 1])
            auc.append(metrics.auc(fpr, tpr))
            pr_auc.append(metrics.average_precision_score(y_test, y_pred[:, 1]))
            penalty.append(p)
            reg.append(c)

auc_df_lr = pd.DataFrame({'penalty' : penalty,
                       'C': reg,
                       'auc' : auc})
auc_df_lr.groupby(['penalty', 'C']).agg({'auc' : ['mean', 'std']}).reset_index()

In [ ]:
#### Logistic regression with regularization grid search + SMOTE
auc = []
penalty = []
reg = []
pr_auc = []

for train_index, test_index in skf.split(X,y):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    y_train, y_test = y[train_index], y[test_index]
    
    # Upsample minority class in training data using SMOTE
    X_train, y_train = SMOTE(random_state = 123).fit_resample(X_train, y_train)
    
    for p in ['l1', 'l2']:
        for c in [10**i for i in range(-3, 3)]:
            lr = LogisticRegression(penalty = p, C = c)
            lr.fit(X_train, y_train)
            y_pred = lr.predict_proba(X_test)
            fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred[:, 1])
            auc.append(metrics.auc(fpr, tpr))
            pr_auc.append(metrics.average_precision_score(y_test, y_pred[:, 1]))
            penalty.append(p)
            reg.append(c)

auc_df_lr = pd.DataFrame({'penalty' : penalty,
                       'C': reg,
                       'auc' : auc})
auc_df_lr.groupby(['penalty', 'C']).agg({'auc' : ['mean', 'std']}).reset_index()

#### 4.2 Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

auc = []
n_tree = []
max_features = []
criterion = []
max_depth = []

for train_index, test_index in skf.split(X,y):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    y_train, y_test = y[train_index], y[test_index]
    
    # Upsample minority class in training data
    X_close = X_train.iloc[y_train == 1, :]
    X_open = X_train.iloc[y_train == 0, :]
    y_close = y_train[y_train == 1]
    y_open = y_train[y_train == 0]
    X_close, y_close = resample(X_close, y_close, n_samples = len(X_open), random_state = 123)
    X_train = pd.concat([X_close, X_open])
    y_train = np.concatenate((y_close, y_open))
    
    for n in [50, 100, 150, 200]:
        for m in ['sqrt', 'log2']:
            for c in ['gini', 'entropy']:
                for d in [int(d) for d in np.linspace(10, 100, num = 10)]:
                    rf = RandomForestClassifier(n_estimators = n, max_features = m, criterion = c, max_depth = d)
                    rf.fit(X_train, y_train)
                    y_pred = rf.predict_proba(X_test)
                    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred[:, 1])
                    auc.append(metrics.auc(fpr, tpr))
                    n_tree.append(n)
                    max_features.append(m)
                    criterion.append(c)
                    max_depth.append(d)

In [ ]:
pd.set_option("display.max_rows", 500)
auc_df_rf = pd.DataFrame({'n_tree' : n_tree,
                       'max_features': max_features,
                       'criterion' : criterion,
                       'max_depth' : max_depth,
                       'auc' : auc})
auc_df_rf.groupby(['n_tree', 'max_features', 'criterion', 'max_depth']).agg({'auc' : ['mean', 'std']}).reset_index()

In [ ]:
auc = []
n_tree = []
max_features = []
criterion = []
max_depth = []

for train_index, test_index in skf.split(X,y):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    y_train, y_test = y[train_index], y[test_index]
    
    # Upsample minority class in training data using SMOTE
    X_train, y_train = SMOTE(random_state = 123).fit_resample(X_train, y_train)
    
    for n in [50, 100, 150, 200]:
        for m in ['sqrt', 'log2']:
            for c in ['gini', 'entropy']:
                for d in [int(d) for d in np.linspace(10, 100, num = 10)]:
                    rf = RandomForestClassifier(n_estimators = n, max_features = m, criterion = c, max_depth = d)
                    rf.fit(X_train, y_train)
                    y_pred = rf.predict_proba(X_test)
                    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred[:, 1])
                    auc.append(metrics.auc(fpr, tpr))
                    n_tree.append(n)
                    max_features.append(m)
                    criterion.append(c)
                    max_depth.append(d)

In [ ]:
auc_df_rf = pd.DataFrame({'n_tree' : n_tree,
                       'max_features': max_features,
                       'criterion' : criterion,
                       'max_depth' : max_depth,
                       'auc' : auc})
auc_df_rf.groupby(['n_tree', 'max_features', 'criterion', 'max_depth']).agg({'auc' : ['mean', 'std']}).reset_index()

#### 4.3 Gradient Boosting

In [ ]:
from xgboost import XGBClassifier
from sklearn.utils import resample
from sklearn import metrics

auc = []
n_tree = []
learning_rate = []
max_depth = []
gamma = []

for train_index, test_index in skf.split(X,y):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    y_train, y_test = y[train_index], y[test_index]
    
    # Upsample minority class in training data
    X_close = X_train.iloc[y_train == 1, :]
    X_open = X_train.iloc[y_train == 0, :]
    y_close = y_train[y_train == 1]
    y_open = y_train[y_train == 0]
    X_close, y_close = resample(X_close, y_close, n_samples = len(X_open), random_state = 123)
    X_train = pd.concat([X_close, X_open])
    y_train = np.concatenate((y_close, y_open))
    
    for n in [50, 100, 150, 200]:
        for i in [0.001, 0.01, 0.1, 0.2]:
            xgb = XGBClassifier(n_estimators = n, learning_rate = i)
            xgb.fit(X_train, y_train)
            y_pred = xgb.predict_proba(X_test)
            fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred[:, 1])
            auc.append(metrics.auc(fpr, tpr))
            n_tree.append(n)
            learning_rate.append(i)    

In [ ]:
pd.set_option("display.max_rows",500)
auc_df_xgb = pd.DataFrame({'n_tree' : n_tree,
                           'learning_rate' : learning_rate,
                           'auc' : auc})
auc_df_xgb.groupby(['n_tree', 'learning_rate']).agg({'auc' : ['mean', 'std']}).reset_index()

In [26]:
from xgboost import XGBClassifier

auc = []
n_tree = []
learning_rate = []
max_depth = []
gamma = []

for train_index, test_index in skf.split(X,y):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    y_train, y_test = y[train_index], y[test_index]
    X_test = X_test.as_matrix()
    
    # Upsample minority class in training data using SMOTE
    X_train, y_train = SMOTE(random_state = 123).fit_resample(X_train, y_train)
    
    for n in [50, 100, 150, 200]:
        for i in [0.001, 0.01, 0.1, 0.2]:
            xgb = XGBClassifier(n_estimators = n, learning_rate = i)
            xgb.fit(X_train, y_train)
            y_pred = xgb.predict_proba(X_test)
            fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred[:, 1])
            auc.append(metrics.auc(fpr, tpr))
            n_tree.append(n)
            learning_rate.append(i)    

In [27]:
pd.set_option("display.max_rows",500)
auc_df_xgb = pd.DataFrame({'n_tree' : n_tree,
                           'learning_rate' : learning_rate,
                           'auc' : auc})
auc_df_xgb.groupby(['n_tree', 'learning_rate']).agg({'auc' : ['mean', 'std']}).reset_index()

n_tree learning_rate       auc          
                             mean       std
0      50         0.001  0.606251  0.014326
1      50         0.010  0.628908  0.011074
2      50         0.100  0.655569  0.008712
3      50         0.200  0.655005  0.012898
4     100         0.001  0.612367  0.008634
5     100         0.010  0.637608  0.009196
6     100         0.100  0.652436  0.014568
7     100         0.200  0.648919  0.008475
8     150         0.001  0.612825  0.008647
9     150         0.010  0.641870  0.004790
10    150         0.100  0.649725  0.012645
11    150         0.200  0.643999  0.009858
12    200         0.001  0.615092  0.008874
13    200         0.010  0.646313  0.003311
14    200         0.100  0.647530  0.013670
15    200         0.200  0.640547  0.013710